# Example 1: Grab metadata from a database

In this example, we will learn how to get metadata of each file in a database.
The metadata contains objective information (e.g. recording date, duration, etc.)
as well as subjective information such as tags.

## List database IDs

Firstly, let's import V4DBHandler from the toolkit to list the ID of databases in `example_db`.

In [1]:
from pydtk.db import V4DBHandler as DBHandler

db_id_handler = DBHandler(db_class='database_id', db_host='./example_db')
db_id_handler.df

,database_id,df_name,_creation_time,_uuid,_id
0,default,db_0ffc6dbe_meta,1.614346e+09,c21f969b5f03d33d43e04f8f136e7682,d318b320783511eb8dd0acde48001122


You can see that `example_db` contains a database with ID `default`.


## Grab a DataFrame of metadata from a database

Now, let's initialize another DBHandler to retrieve the metadata of the contents in `default` database.

In [2]:
from pydtk.db import V4DBHandler as DBHandler

db_handler = DBHandler(
    db_class='meta',
    db_host='./example_db',
    database_id='default',
    base_dir_path='../test',
    read_on_init=True,
    orient='contents'
)

If you set `read_on_init` to `True`, the entire contents in the database will be loaded
and stored into the local memory as a Pandas DataFrame.  
You can access to the contents as follows.

In [3]:
db_handler.df

,description,database_id,record_id,sub_record_id,data_type,path,start_timestamp,end_timestamp,content_type,contents,msg_type,msg_md5sum,count,frequency,tags,_creation_time,_uuid,_id
0,Driving Database,Driving Behavior Database,016_00000000030000000240,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/016_0...,1.489728e+09,1.489729e+09,text/csv,camera/front-center,NaN,NaN,NaN,NaN,"[camera, front, center, timestamps]",1.614346e+09,604a772a56fff3c02c5dd159eb20c305,d9f73cc0783511eb8dd0acde48001122
1,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/acceleration,geometry_msgs/AccelStamped,d8a98a5d81351b6eb0578c78557e7659,10.0,10.000010,"[vehicle, acceleration]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
2,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/analog/back_signal,std_msgs/Bool,8b94c1b53db61fb6aed406028ad6332a,20.0,20.000019,"[vehicle, analog, back, signal]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
3,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/analog/brake_signal,std_msgs/Bool,8b94c1b53db61fb6aed406028ad6332a,20.0,20.000019,"[vehicle, analog, brake, signal]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
4,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/analog/speed_pulse,std_msgs/UInt8,7c8164229e7d2c17eb95e9231617fdee,20.0,20.000019,"[vehicle, analog, speed, pulse]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
5,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/analog/turn_signal,std_msgs/UInt8,7c8164229e7d2c17eb95e9231617fdee,20.0,20.000019,"[vehicle, analog, turn, signal]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
6,Driving Database,Driving Behavior Database,B05_17000000010000000829,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/B05_1...,1.517463e+09,1.517463e+09,application/rosbag,/vehicle/gnss,sensor_msgs/NavSatFix,2d3a8cd499b9b4a0249fb98fd05cfa48,1.0,NaN,"[vehicle, gnss]",1.614346e+09,da731b0a6b80b04c252a7abde02e86ba,d9f7651a783511eb8dd0acde48001122
7,Driving Database,Driving Behavior Database,sample,NaN,raw_data,/Users/d_hayashi/gits/pydtk/test/records/sampl...,1.550126e+09,1.550126e+09,application/rosbag,/points_concat_downsampled,sensor_msgs/PointCloud2,1158d486dd51d683ce2f1be655c3c181,4.0,10.000000,"[lidar, downsampled]",1.614346e+09,64e8601dc0361b0e59b9fa51ee89070b,d9f781f8783511eb8dd0acde48001122
8,Urban driving situation description.,METI2019,20191001_094731_000_car3,20191001_094731_car3_101732,raw_data,/Users/d_hayashi/gits/pydtk/test/records/meti2...,1.569893e+09,1.569893e+09,application/rosbag,/vehicle/can_raw,autoware_can_msgs/CANPacket,8315bda71683b8ece50e17e529eea4c1,698217.0,4173.436816,"[vehicle, can, raw]",1.614346e+09,0dc27d8c7a72eca3b5fec99d4d0ee813,d9f79f1c783511eb8dd0acde48001122
9,JERA Forecast,JERA Forecast,test,NaN,forecast,/Users/d_hayashi/gits/pydtk/test/records/forec...,NaN,NaN,text/csv,forecast,NaN,NaN,NaN,NaN,"[test1, test2]",1.614346e+09,7fa313848e0dca92bccd48846498bd87,d9f7bd30783511eb8dd0acde48001122


When you want to handle a very large dataset, the metadata contains huge amount of information and as a result,
it takes a long time to load all of it.  
However, if you want to grab only a limited scope (e.g. metadata of files tagged 'camera' and 'front'),
it is costful to load all the dataset and search items on the loaded dataframe.  
Therefore, the toolkit provides a method to execute a query before loading the database
and limit the items to load.  

To execute a query before loading metadata, you should set `read_on_init` option to `False` as follows.

V4DBHandler supports the DB-native queries and [PQL](https://github.com/alonho/pql).

In [4]:
# Initialize DB-handler
db_handler = DBHandler(
    db_class='meta',
    db_host='./example_db',
    database_id='default',
    base_dir_path='../test',
    read_on_init=False,
    orient='contents'
)

# Filter records by the timestamps
db_handler.read(pql='start_timestamp > 1500000000')
print('# of metadata: {}'.format(len(db_handler.df)))
db_handler.read(pql='start_timestamp > 1500000000 and end_timestamp < 1520000000')
print('# of metadata: {}'.format(len(db_handler.df)))

# Filter records by `record_id` with regular expressions
db_handler.read(pql='record_id == regex("B05.*")')
print('# of metadata: {}'.format(len(db_handler.df)))

# Read metadata containing a specific key
db_handler.read(pql='"contents./points_concat_downsampled" != ""')
print('# of metadata: {}'.format(len(db_handler.df)))

# You can also use DB-native queries (Tinymongo is used in this case)
db_handler.read(query={'start_timestamp': {'$gt': 1500000000}})
print('# of metadata: {}'.format(len(db_handler.df)))
db_handler.read(query={'$and': [{'start_timestamp': {'$gt': 1500000000}}, {'end_timestamp': {'$lt': 1520000000}}]})
print('# of metadata: {}'.format(len(db_handler.df)))
db_handler.read(query={'record_id': {'$regex': 'B05.*'}})
print('# of metadata: {}'.format(len(db_handler.df)))
db_handler.read(query={'contents./points_concat_downsampled': {'$ne': ''}})
print('# of metadata: {}'.format(len(db_handler.df)))

# of metadata: 8
# of metadata: 6
# of metadata: 6
# of metadata: 1
# of metadata: 8
# of metadata: 6
# of metadata: 6
# of metadata: 1


## Get list for record_id corresponding to metadata

Each row of the dataframe acquired above corresponds to a file in the dataset.  
If you wan to know which record-id the file belongs to, you can get a dataframe of records as follows.

In [5]:
db_handler.record_id_df

,record_id,start_timestamp,end_timestamp,tags,duration
0,sample,1.550126e+09,1.550126e+09,"[lidar, downsampled]",0.31


You can get list of contents as well.

In [6]:
db_handler.content_df



,record_id,path,content,msg_type,tag
0,sample,/Users/d_hayashi/gits/pydtk/test/records/sampl...,/points_concat_downsampled,sensor_msgs/PointCloud2,"[lidar, downsampled]"
